In [4]:
import pandas as pd
import folium
from pyproj import crs
import geopandas as gpd
import matplotlib.pyplot as plt
import requests
import time

In [5]:
file_path = 'winemag-data-130k-v2.csv'
wine_data = pd.read_csv(file_path)

cleaned_data = wine_data.drop(columns=['region_1', 'region_2', 'taster_name', 'taster_twitter_handle'])
cleaned_data = cleaned_data.dropna()

cleaned_file_path = 'preprocessed_winemag-data.csv'
cleaned_data.to_csv(cleaned_file_path, index=False)

In [6]:
preprocessed_file = "preprocessed_winemag-data.csv"
preprocessed_data = pd.read_csv(preprocessed_file)
total_provinces = preprocessed_data['country'].nunique()
print(total_provinces)

41


In [14]:
lausanne = [46.5197, 6.6323]
my_map = folium.Map(location=lausanne, zoom_start=4, control_scale=True,tiles='https://api.mapbox.com/styles/v1/sinanulcay/clvz14wkt02bs01qv0m9v611f/tiles/256/{z}/{x}/{y}@2x?access_token=pk.eyJ1Ijoic2luYW51bGNheSIsImEiOiJUVDNPX25NIn0.U1Y_zG52ui-QSLQ5BZvY6w', attr='Mapbox Light')

In [15]:
my_map

In [16]:
outfp = "base_layer.html"
my_map.save(outfp)
filtered_file = "winery_data.csv"
filtered_data = pd.read_csv(filtered_file)
filtered_data['province_country'] = filtered_data['province'] + '+' + filtered_data['country']
filtered_data_path = 'winery_data.csv'
filtered_data.to_csv(filtered_data_path, index=False)

In [17]:
total_provinces = filtered_data['province_country'].nunique()
print(total_provinces)

276


In [18]:

base_url = "https://geocode.maps.co/search"
query_string = "California+US"
api_key = "6623e0c3016bc741318638ixybbbfbc"

full_url = f"{base_url}?q={query_string}&api_key={api_key}"
response = requests.get(full_url)

if response.status_code == 200:
    data = response.json()
    print(data)
    latitude = data[0]['lat']
    longitude = data[0]['lon']
else:
    print(f"Failed to retrieve data: {response.status_code}")

[{'place_id': 285364073, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'relation', 'osm_id': 165475, 'boundingbox': ['32.5295236', '42.009499', '-124.482003', '-114.1307816'], 'lat': '36.7014631', 'lon': '-118.755997', 'display_name': 'California, United States', 'class': 'boundary', 'type': 'administrative', 'importance': 0.9221363841577066}, {'place_id': 298700331, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'relation', 'osm_id': 141178, 'boundingbox': ['38.616032', '38.645932', '-92.595475', '-92.524332'], 'lat': '38.628683', 'lon': '-92.5659635', 'display_name': 'California, Moniteau County, Missouri, 65018, United States', 'class': 'boundary', 'type': 'administrative', 'importance': 0.5231335668641502}, {'place_id': 275328038, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'node', 'osm_id': 6977830482, 'boundingbox': ['13.4025234

In [19]:
base_url = "https://geocode.maps.co/search"
api_key = "6623e0c3016bc741318638ixybbbfbc"

filtered_data['lat'] = None
filtered_data['lon'] = None

unique_locations = filtered_data['province_country'].unique()

location_dict = {}

for location in unique_locations:
    full_url = f"{base_url}?q={location}&api_key={api_key}"

    response = requests.get(full_url)
    time.sleep(1) 

    if response.status_code == 200:
        data = response.json()
        if data:
            latitude = data[0]['lat']
            longitude = data[0]['lon']
            location_dict[location] = (latitude, longitude)
        else:
            print(f"No results for {location}")
    else:
        print(f"Failed to retrieve data for {location}: {response.status_code}")


for index, row in filtered_data.iterrows():
    location = row['province_country']
    if location in location_dict:
        filtered_data.at[index, 'lat'] = location_dict[location][0]
        filtered_data.at[index, 'lon'] = location_dict[location][1]

filtered_data_path = 'winery_data.csv'
filtered_data.to_csv(filtered_data_path, index=False)



KeyboardInterrupt: 

In [ ]:
empty_lat_lon_count = filtered_data[filtered_data['lat'].isna() | filtered_data['lon'].isna()].shape[0]
print(empty_lat_lon_count)

0


In [ ]:
filtered_data = filtered_data.dropna(subset=['lat', 'lon'])

In [ ]:
filtered_data_path = 'winery_data.csv'
filtered_data.to_csv(filtered_data_path, index=False)

In [ ]:
lausanne = [46.5197, 6.6323]
my_map = folium.Map(location=lausanne, zoom_start=4, control_scale=True)

In [20]:
import folium
import pandas as pd
from folium.plugins import MarkerCluster
import numpy as np

filtered_data = pd.read_csv('winery_data.csv')
winery_data = filtered_data

def add_random_noise(lat, lon, radius_km=1):
    degree_change = radius_km / 111  
    new_lat = lat + np.random.uniform(-degree_change, degree_change)
    new_lon = lon + np.random.uniform(-degree_change, degree_change)
    return new_lat, new_lon

lausanne = [46.5197, 6.6323]
map = folium.Map(location=lausanne, zoom_start=6,tiles='https://api.mapbox.com/styles/v1/sinanulcay/clvz14wkt02bs01qv0m9v611f/tiles/256/{z}/{x}/{y}@2x?access_token=pk.eyJ1Ijoic2luYW51bGNheSIsImEiOiJUVDNPX25NIn0.U1Y_zG52ui-QSLQ5BZvY6w', attr='Mapbox Light')

marker_cluster = MarkerCluster().add_to(map)
used_coords = set()

for idx, row in winery_data.iterrows():
    lat, lon = row['lat'], row['lon']

    while (lat, lon) in used_coords:
        lat, lon = add_random_noise(float(lat), float(lon))
    
    used_coords.add((lat, lon))
    
    popup_message = folium.Popup(f"<b>Winery:</b> {row['winery']}<br>"
                                 f"<b>Mean winery score:</b> {row['mean']}<br>"
                                 f"<b>Higher ranked wine score:</b> {row['max']}<br>"
                                 f"<b>Higher ranked wine:</b> {row['title']}<br>"
                                 f"<b>Higher ranked price in dollars:</b> {row['price']}<br>"
                                 f"<b>Wine variety:</b> {row['variety']}", 
                                 max_width=300)
    
    folium.Marker(
        location=[lat, lon],
        popup=popup_message
    ).add_to(marker_cluster)

map.save('winery_map_no_overlap_1km.html')
